In [1]:
# Manipular grandes volúmenes de datos utilizando Spark SQL para
# resolver un problema

# Machine learning para clasificar
# Python es el lenguaje preferido para la ciencia de datos debido a NumPy,
# Pandas y matplotlib, que son herramientas que facilitan el trabajo con
# matrices y dibujan gráficos y pueden trabajar con grandes matrices de datos
# de manera eficiente. Pero Spark está diseñado para trabajar con una enorme
# cantidad de datos, distribuidos en un clúster.

# Datos del paciente del corazón
# Encontrará adjunto la base de datos.
# Las columnas son:
# 1. Años
# 2. Sexo
# 3. Tipo de dolor torácico (4 valores)
# 4. Presión arterial en reposo
# 5. Colesterol sérico en mg/dl
# 6. Azúcar en sangre en ayunas > 120 mg/dl
# 7. Resultados electrocardiográficos en reposo (valores 0,1,2)
# 8. Frecuencia cardíaca máxima alcanzada
# 9. Angina inducida por el ejercicio
# 10. Oldpeak = depresión del ST inducida por el ejercicio en relación con el reposo
# 11. Pendiente del segmento ST de ejercicio máximo
# 12. Número de vasos principales (0-3) coloreados por fluoroscopia
# 13. Thal: 3 = normal; 6 = defecto fijo; 7 = defecto reversible

# El campo que indica si el paciente tiene un problema cardíaco. Los números
# son los siguientes:

# Un valor de 3 significa que el paciente está sano (normal). Un valor de 6
# significa que se ha solucionado el problema de salud del paciente. Un valor de
# 7 significa que se puede solucionar el problema de salud del paciente.




In [2]:
# importado de librerias
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import IntegerType

# Crear una configuración de Spark
conf = SparkConf().setAppName("MyApp").setMaster("local[*]")

sc = SparkContext(conf=conf)

from pyspark.sql import SparkSession
spark = SparkSession(sc)


# Cargar el dataset con esta definicion de columnas y tipos

df = spark.read.csv("heart.dat", header=False, sep=' ', inferSchema=True)

# set column names
schema = ['Años','Sexo','Tipo_dolor_toracico','Presion_arterial_reposo','Colesterol_serico','Azucar_sangre_ayunas',
            'Resultados_electrocardiográficos_reposo', 'Frecuencia_cardíaca_máxima', 'Angina_inducida_ejercicio',
            'Oldpeak', 'Pendiente_segmento ST','Número_vasos_principales','Thal','Target']
for i in range(len(schema)):
    col = '_c'+str(i) # nombre de la columna
    #df = df.withColumn(col, df[col].cast(FloatType())) # convertir a FloatType
    df = df.withColumnRenamed(col, schema[i]) # renombrar columnas
    
# drop column Target
df = df.drop('Target')

   

# Ver el esquema de los datos
df.printSchema()

# Ver los datos
df.show()



root
 |-- Años: double (nullable = true)
 |-- Sexo: double (nullable = true)
 |-- Tipo_dolor_toracico: double (nullable = true)
 |-- Presion_arterial_reposo: double (nullable = true)
 |-- Colesterol_serico: double (nullable = true)
 |-- Azucar_sangre_ayunas: double (nullable = true)
 |-- Resultados_electrocardiográficos_reposo: double (nullable = true)
 |-- Frecuencia_cardíaca_máxima: double (nullable = true)
 |-- Angina_inducida_ejercicio: double (nullable = true)
 |-- Oldpeak: double (nullable = true)
 |-- Pendiente_segmento ST: double (nullable = true)
 |-- Número_vasos_principales: double (nullable = true)
 |-- Thal: double (nullable = true)

+----+----+-------------------+-----------------------+-----------------+--------------------+---------------------------------------+--------------------------+-------------------------+-------+---------------------+------------------------+----+
|Años|Sexo|Tipo_dolor_toracico|Presion_arterial_reposo|Colesterol_serico|Azucar_sangre_ayunas|Res

In [3]:
# Entonces, escriba esta función ENFERMO() para marcar 0 como negativo y 1
# como positivo, porque la regresión logística binaria requiere uno de dos
# resultados.

# Importar librerias
from pyspark.sql.functions import udf

# Crear la funcion
def ENFERMO(x):
    if x in(3, 6): # 3 = normal; 6 = problema solucionado
        return 0
    else:
        return 1

# Crear la funcion UDf
udfENFERMO = udf(ENFERMO, IntegerType())

# Aplicar la funcion UDF a la columna target usando el valor de that
df = df.withColumn("Target", udfENFERMO("Thal") )

# Ver los datos
df.show(5)


+----+----+-------------------+-----------------------+-----------------+--------------------+---------------------------------------+--------------------------+-------------------------+-------+---------------------+------------------------+----+------+
|Años|Sexo|Tipo_dolor_toracico|Presion_arterial_reposo|Colesterol_serico|Azucar_sangre_ayunas|Resultados_electrocardiográficos_reposo|Frecuencia_cardíaca_máxima|Angina_inducida_ejercicio|Oldpeak|Pendiente_segmento ST|Número_vasos_principales|Thal|Target|
+----+----+-------------------+-----------------------+-----------------+--------------------+---------------------------------------+--------------------------+-------------------------+-------+---------------------+------------------------+----+------+
|70.0| 1.0|                4.0|                  130.0|            322.0|                 0.0|                                    2.0|                     109.0|                      0.0|    2.4|                  2.0|                  

In [4]:
# También debe crear el dataframe de Spark raw_data usando la operación
# transform() y seleccionando solo la columna de características.

# Importar librerias
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

# set columns except the col Target
columns = df.columns
columns.remove('Target')


# Crear el vector assembler
assembler = VectorAssembler(inputCols=columns, outputCol='features')

# Transformar los datos
raw_data = assembler.transform(df).select("features", "Target")

# Ver los datos
raw_data.show(5)


+--------------------+------+
|            features|Target|
+--------------------+------+
|[70.0,1.0,4.0,130...|     0|
|[67.0,0.0,3.0,115...|     1|
|[57.0,1.0,2.0,124...|     1|
|[64.0,1.0,4.0,128...|     1|
|[74.0,0.0,2.0,120...|     0|
+--------------------+------+
only showing top 5 rows



In [5]:
# 1. Use Standard Scaler para poner todos los números en la misma escala.
# Esto toma la observación y resta la media, y luego la divide por la
# desviación estándar.

# Importar librerias
from pyspark.ml.feature import StandardScaler

# Crear el objeto StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=True)

# Calcular la media y la desviacion estandar
scalerModel = scaler.fit(raw_data) 

# Normalizar los datos
scaledData = scalerModel.transform(raw_data)



In [6]:
# 2. Imprima el estado actual de sus datos

# Ver los datos
scaledData.show(5)


+--------------------+------+--------------------+
|            features|Target|      scaledFeatures|
+--------------------+------+--------------------+
|[70.0,1.0,4.0,130...|     0|[1.70892007713705...|
|[67.0,0.0,3.0,115...|     1|[1.37957787811706...|
|[57.0,1.0,2.0,124...|     1|[0.28177054805043...|
|[64.0,1.0,4.0,128...|     1|[1.05023567909707...|
|[74.0,0.0,2.0,120...|     0|[2.14804300916370...|
+--------------------+------+--------------------+
only showing top 5 rows



In [7]:
# 3. Divida los datos en conjuntos de datos de entrenamiento y prueba. Use 50/50.
train_data, test_data = scaledData.randomSplit([0.5, 0.5])

train_data.show(5)


+--------------------+------+--------------------+
|            features|Target|      scaledFeatures|
+--------------------+------+--------------------+
|(13,[0,2,3,4,7,10...|     0|[-1.9138441120828...|
|(13,[0,2,3,4,7,10...|     0|[-1.4747211800561...|
|(13,[0,2,3,4,7,10...|     0|[-1.3649404470495...|
|(13,[0,2,3,4,7,10...|     0|[-1.0355982480295...|
|(13,[0,2,3,4,7,10...|     0|[-0.9258175150228...|
+--------------------+------+--------------------+
only showing top 5 rows



In [8]:
# 4. Cree un modelo de regresión logística y entrénalo.

# Importar librerias
from pyspark.ml.classification import LogisticRegression

# Crear el objeto LogisticRegression
model = LogisticRegression(labelCol="Target", featuresCol="scaledFeatures")

# Entrenar el modelo
trainedModel = model.fit(train_data)

# Imprima los coeficientes y el intercepto para el modelo de regresión logística
print("Coeficientes: " + str(trainedModel.coefficients))
print("Intercepto: " + str(trainedModel.intercept))



Coeficientes: [-3.4951626823105677,-26.679313464084693,-4.4087383198706105,-0.6424689176541718,9.237879168078045,-1.5592462686678907,-4.8878516394218,7.978132017061151,4.2115804052136,6.492090146151653,-6.673431922172488,-1.1174446075894764,75.03453148392056]
Intercepto: -37.573629977042906


In [9]:
# 5. Finalmente Use la función F de Spark SQL para crear una nueva columna
# correcta cuando ENFERMO() es igual a la predicción, lo que significa que
# el resultado predicho es igual a los resultados reales.

# Importar librerias
from pyspark.sql.functions import col

# Crear la funcion
def CHECK_RESULT(x, y):
    if int(x) == int(y):
        return 1
    else:
        return 0

# Crear la funcion UDf
udfCHECK_RESULT = udf(CHECK_RESULT, IntegerType())

# Crear la columna correcta
predictions = trainedModel.transform(test_data)
predictions = predictions.withColumn("correct", udfCHECK_RESULT("Target", "prediction"))

# Ver los datos
predictions.show(5)

# conteo de datos
print(f"Aciertos: ",predictions.filter(predictions.correct == 1).count()," de ", predictions.count())



+--------------------+------+--------------------+--------------------+--------------------+----------+-------+
|            features|Target|      scaledFeatures|       rawPrediction|         probability|prediction|correct|
+--------------------+------+--------------------+--------------------+--------------------+----------+-------+
|(13,[0,2,3,4,7,10...|     0|[-1.6942826460694...|[51.5153470684896...|           [1.0,0.0]|       0.0|      1|
|(13,[0,2,3,4,7,10...|     0|[0.39155128105709...|[36.9507097314047...|           [1.0,0.0]|       0.0|      1|
|(13,[0,2,3,4,7,10...|     0|[0.83067421308374...|[69.7829005123032...|           [1.0,0.0]|       0.0|      1|
|[29.0,1.0,2.0,130...|     0|[-2.7920899761361...|[102.297827209384...|           [1.0,0.0]|       0.0|      1|
|[34.0,0.0,2.0,118...|     0|[-2.2431863111028...|[35.4083777080133...|[0.99999999999999...|       0.0|      1|
+--------------------+------+--------------------+--------------------+--------------------+----------+-

In [10]:
#cerrar la sesion de spark
spark.stop()


In [11]:
# - SQL Spark

# SQL Spark es un módulo de Spark para el procesamiento de datos estructurados.
# Proporciona una API para manipular datos estructurados utilizando el lenguaje
# SQL estándar. Puede ejecutar consultas SQL escritas en el lenguaje SQL estándar
# utilizando Spark SQL. Spark SQL proporciona una API para manipular datos
# estructurados utilizando el lenguaje SQL estándar. Puede ejecutar consultas SQL
# escritas en el lenguaje SQL estándar utilizando Spark SQL. Spark SQL proporciona
# una API para manipular datos estructurados utilizando el lenguaje SQL estándar.
# Puede ejecutar consultas SQL escritas en el lenguaje SQL estándar utilizando
# Spark SQL.

# - Formatos de Archivo
# Spark SQL proporciona una API para leer archivos de texto, JSON, CSV, parquet,
# ORC y muchos otros formatos de archivo. Puede leer datos de estos formatos y
# crear un DataFrame a partir de ellos. También puede escribir datos en estos
# formatos.


# - Machine Learning con Spark
# Spark proporciona una API para el aprendizaje automático llamada MLlib. Puede
# usar MLlib para crear modelos de aprendizaje automático. MLlib proporciona
# algoritmos de aprendizaje automático como clasificación, regresión, agrupación,
# filtrado colaborativo, reducción de dimensionalidad, así como tuberías de
# aprendizaje automático y extracción de características. MLlib también admite
# herramientas de terceros como TensorFlow. Puede usar TensorFlow para crear
# modelos de aprendizaje automático y usar MLlib para entrenarlos y usarlos para
# predicciones.

# - MLlib
# MLlib es la biblioteca de aprendizaje automático de Spark. Proporciona
# algoritmos de aprendizaje automático como clasificación, regresión, agrupación,
# filtrado colaborativo, reducción de dimensionalidad, así como tuberías de
# aprendizaje automático y extracción de características. MLlib también admite
# herramientas de terceros como TensorFlow. Puede usar TensorFlow para crear
# modelos de aprendizaje automático y usar MLlib para entrenarlos y usarlos para
# predicciones.
